In [1]:
!pip install openai

In [2]:
!pip install google-api-python-client

In [3]:
import openai
import json
from google.oauth2 import service_account
import googleapiclient.discovery

In [4]:
# Load and set our OpenAI API key
openai.api_key = open("key.txt", "r").read().strip("\n")

In [5]:
# Load and set Google Calendar API credentials
credentials = service_account.Credentials.from_service_account_file('YOUR CREDENTIALS FILE.json')
calendar_service = googleapiclient.discovery.build('calendar', 'v3', credentials=credentials)

In [6]:
def get_calendar_events(date):
    """Get the events on a given date from Google Calendar"""
    events_result = calendar_service.events().list(
        calendarId='primary',
        timeMin=f'{date}T00:00:00Z',
        timeMax=f'{date}T23:59:59Z',
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])
    formatted_events = []
    for event in events:
        formatted_event = {
            "summary": event['summary'],
            "start_time": event['start'].get('dateTime', event['start'].get('date')),
            "location": event.get('location', 'N/A'),
        }
        formatted_events.append(formatted_event)

    return json.dumps({"date": date, "events": formatted_events})

In [7]:
def create_event(summary, start_time, end_time, location=None):
    """Create a new event in Google Calendar"""
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'UTC',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'UTC',
        },
    }

    if location:
        event['location'] = location

    created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
    return json.dumps({"event_id": created_event['id'], "summary": created_event['summary'], "status": "created"})

In [8]:
def delete_event(event_id):
    """Delete an event from Google Calendar"""
    calendar_service.events().delete(calendarId='primary', eventId=event_id).execute()
    return json.dumps({"event_id": event_id, "status": "deleted"})

In [9]:
def get_free_busy_info(start_time, end_time):
    """Get free/busy information from Google Calendar"""
    free_busy_query = {
        "timeMin": start_time,
        "timeMax": end_time,
        "items": [{"id": "primary"}],
    }

    free_busy_info = calendar_service.freebusy().query(body=free_busy_query).execute()
    return json.dumps(free_busy_info)

In [10]:
def update_event(event_id, summary=None, start_time=None, end_time=None, location=None):
    """Update an event in Google Calendar"""
    event = calendar_service.events().get(calendarId='primary', eventId=event_id).execute()

    if summary:
        event['summary'] = summary
    if start_time:
        event['start']['dateTime'] = start_time
    if end_time:
        event['end']['dateTime'] = end_time
    if location:
        event['location'] = location

    updated_event = calendar_service.events().update(calendarId='primary', eventId=event_id, body=event).execute()
    return json.dumps({"event_id": updated_event['id'], "summary": updated_event['summary'], "status": "updated"})


In [11]:
def reschedule_event(event_id, start_time, end_time):
    """Reschedule an event in Google Calendar"""
    return update_event(event_id, start_time=start_time, end_time=end_time)

In [12]:
# Function definitions for calendar-related queries
functions = [
    {
        "name": "get_calendar_events",
        "description": "Get the events on a given date from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "The date for which to retrieve events (in 'YYYY-MM-DD' format)",
                },
            },
            "required": ["date"],
        },
    },
    {
        "name": "create_event",
        "description": "Create a new event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "The summary or title of the event",
                },
                "start_time": {
                    "type": "string",
                    "description": "The start time of the event (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The end time of the event (in RFC3339 format)",
                },
                "location": {
                    "type": "string",
                    "description": "The location of the event (optional)",
                },
            },
            "required": ["summary", "start_time", "end_time"],
        },
    },
    {
        "name": "delete_event",
        "description": "Delete an event from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be deleted",
                },
            },
            "required": ["event_id"],
        },
    },
    {
        "name": "get_free_busy_info",
        "description": "Get free/busy information from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "start_time": {
                    "type": "string",
                    "description": "The start time for querying free/busy information (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The end time for querying free/busy information (in RFC3339 format)",
                },
            },
            "required": ["start_time", "end_time"],
        },
    },
    {
        "name": "update_event",
        "description": "Update an event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be updated",
                },
                "summary": {
                    "type": "string",
                    "description": "The new summary or title of the event (optional)",
                },
                "start_time": {
                    "type": "string",
                    "description": "The new start time of the event (in RFC3339 format) (optional)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The new end time of the event (in RFC3339 format) (optional)",
                },
                "location": {
                    "type": "string",
                    "description": "The new location of the event (optional)",
                },
            },
            "required": ["event_id"],
        },
    },
    {
        "name": "reschedule_event",
        "description": "Reschedule an event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be rescheduled",
                },
                "start_time": {
                    "type": "string",
                    "description": "The new start time of the event (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The new end time of the event (in RFC3339 format)",
                },
            },
            "required": ["event_id", "start_time", "end_time"],
        },
    },
]


In [13]:
# Chat with the personalized calendar chatbot
messages = [
    {"role": "user", "content": "What events do I have?"},
    {"role": "assistant", "content": 'I will check your calendar for events.'},
    {"role": "assistant", "content": 'Assistant to=functions.get_calendar_events'},
    {"role": "user", "content": "Tell me about my events."},
    {"role": "assistant", "content": "You don't have any events scheduled."},
    {"role": "user", "content": "Create a new event"},
    {"role": "assistant", "content": 'Assistant to=functions.create_event'},
    {"role": "user", "content": "Delete the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.delete_event'},
    {"role": "user", "content": "Get free/busy information"},
    {"role": "assistant", "content": 'Assistant to=functions.get_free_busy_info'},
    {"role": "user", "content": "Update the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.update_event'},
    {"role": "user", "content": "Reschedule the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.reschedule_event'},
]

In [14]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    max_tokens=50,
)

In [15]:
reply_content = completion.choices[0]
reply_content

<OpenAIObject at 0x1d02fbf2810> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "{\n  \"date\": \"2022-05-20\"\n}",
    "function_call": {
      "name": "get_calendar_events",
      "arguments": "{\n  \"date\": \"2022-05-20\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [ ]:
import openai
import json

# Set the OpenAI API key
openai.api_key = "YOUR API KEY"

def process_user_message(user_message, context):
    if user_message:
        # Append the user message to the conversation context
        context.append({"role": "user", "content": user_message})

        # Generate a response from the chatbot
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=context,
            functions=functions,
            max_tokens=100,
        )

        # Retrieve the assistant's reply from the response
        assistant_reply = response['choices'][0]['message']['content']

        # Append the assistant's reply to the conversation context
        context.append({"role": "assistant", "content": assistant_reply})

        return assistant_reply, context
    else:
        return "", context

# Define the initial conversation context
conversation_context = [
    {"role": "system", "content": "You are a chatbot that can interact with Google Calendar."},
    {"role": "system", "content": "You can ask me to create, delete, update, or reschedule events."},
    {"role": "system", "content": "You can also get information about your events and free/busy status."},
    {"role": "user", "content": "What events do I have?"},
]

# Start the conversation loop
while True:
    user_input = input("User: ")

    # Process the user's message and get the assistant's reply
    assistant_response, conversation_context = process_user_message(user_input, conversation_context)

    # Print the assistant's reply
    print("Assistant:", assistant_response)

    # Check if the conversation has ended
    if assistant_response == "Goodbye!":
        break

User: Hi
Assistant: Hello! How can I assist you today?
User: Can you create a new event in Google Calendar?
Assistant: Of course! Please provide me with the necessary details for the event.
User: Which details you want?
Assistant: To create a new event in Google Calendar, I need the following details:

1. Summary or title of the event
2. Start time of the event (in RFC3339 format)
3. End time of the event (in RFC3339 format)
4. Location of the event (optional)

Please provide these details so that I can create the event for you.
User: 1. Meeting for capstone project. Start time is 2023-07-11T07:20:50.52Z and End time is 2023-7-11T08:20:50.52Z
Assistant: None
